Download and Extract the data set

In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "akmalhussain72"

os.environ['KAGGLE_KEY'] ="0497cdcb67183b7a904f124f92eda0c1"

!kaggle datasets download -d fpeccia/weed-detection-in-soybean-crops

100% 2.36G/2.37G [00:22<00:00, 75.9MB/s]
100% 2.37G/2.37G [00:22<00:00, 114MB/s] 


In [ ]:

 
#ZipFile is a class of zipfile module for reading and writing zip files
from zipfile import ZipFile
#filename = zip file path 
file_name = "/content/weed-detection-in-soybean-crops.zip"
#to extract the zip file
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
pwd

'/content'

Making The Directories

In [ ]:
os.mkdir("/content/train_data")

In [ ]:
os.mkdir("/content/train_data/broadleaf")

In [ ]:
os.mkdir("/content/train_data/grass")

In [ ]:
os.mkdir("/content/train_data/soil")

In [ ]:
os.mkdir("/content/train_data/soybean")

In [ ]:
os.mkdir("/content/test_data")

In [ ]:
os.mkdir("/content/test_data/broadleaf")

In [ ]:
os.mkdir("/content/test_data/grass")

In [ ]:
os.mkdir("/content/test_data/soil")

In [ ]:
os.mkdir("/content/test_data/soybean")

In [ ]:
import shutil

In [ ]:
from shutil import copyfile 
import random

Split the data

In [ ]:
def split_data(source,training,testing,split_size):
  files = []
  for filename in os.listdir(source):
    file = source+filename
    files.append(filename)
    training_length = int(len(files)*split_size)
    shuffled_set = random.sample(files,len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[training_length:]
  for filename in training_set:
    this_file = source+filename 
    destination = training+filename 
    copyfile(this_file,destination)        
  for filename in testing_set:
    this_file = source+filename 
    destination = testing+filename 
    copyfile(this_file,destination)
# splitt data function and size is o.8
split_data("/content/dataset/broadleaf/","/content/train_data/broadleaf/","/content/test_data/broadleaf/",0.8)
split_data("/content/dataset/grass/","/content/train_data/grass/","/content/test_data/grass/",0.8)
split_data("/content/dataset/soil/","/content/train_data/soil/","/content/test_data/soil/",0.8)
split_data("/content/dataset/soybean/","/content/train_data/soybean/","/content/test_data/soybean/",0.8)


Data splitted

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

Using TensorFlow backend.


In [ ]:
train_dir = '/content/train_data'
test_dir = '/content/test_data'

In [ ]:
train_gen_obj = ImageDataGenerator(rescale = 1.0/255.0)
test_gen_obj = ImageDataGenerator(rescale = 1.0/255.0)

In [ ]:
train_gen = train_gen_obj.flow_from_directory(train_dir, batch_size = 128, target_size = (150, 150), class_mode = 'categorical')
test_gen = test_gen_obj.flow_from_directory(test_dir, batch_size = 64, target_size = (150, 150), class_mode = 'categorical')

Found 12267 images belonging to 4 classes.
Found 3069 images belonging to 4 classes.


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dense(4, activation='softmax')  
    ])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        1

In [ ]:
# Compile Model. 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
#Train the Model
history = model.fit_generator(train_gen,
                              validation_data = test_gen,
                              epochs = 10, verbose = 1)

Epoch 1/10
192/192 [==============================] - 67s 348ms/step - loss: 0.5246 - acc: 0.7944 - val_loss: 0.4456 - val_acc: 0.8804
Epoch 2/10
192/192 [==============================] - 60s 313ms/step - loss: 0.3010 - acc: 0.8885 - val_loss: 0.5729 - val_acc: 0.8521
Epoch 3/10
192/192 [==============================] - 60s 314ms/step - loss: 0.2345 - acc: 0.9145 - val_loss: 0.2687 - val_acc: 0.9176
Epoch 4/10
192/192 [==============================] - 61s 316ms/step - loss: 0.1661 - acc: 0.9398 - val_loss: 0.2689 - val_acc: 0.9312
Epoch 5/10
192/192 [==============================] - 60s 314ms/step - loss: 0.1296 - acc: 0.9506 - val_loss: 0.1752 - val_acc: 0.9303
Epoch 6/10
192/192 [==============================] - 60s 315ms/step - loss: 0.0994 - acc: 0.9647 - val_loss: 0.0819 - val_acc: 0.9309
Epoch 7/10
192/192 [==============================] - 60s 314ms/step - loss: 0.0682 - acc: 0.9764 - val_loss: 0.4128 - val_acc: 0.9407
Epoch 8/10
192/192 [==============================] - 60s 310ms/step - loss: 0.0512 - acc: 0.9824 - val_loss: 0.5732 - val_acc: 0.9339
Epoch 9/10
192/192 [==============================] - 60s 313ms/step - loss: 0.0401 - acc: 0.9857 - val_loss: 0.3409 - val_acc: 0.9185
Epoch 10/10
192/192 [==============================] - 61s 317ms/step - loss: 0.0291 - acc: 0.9916 - val_loss: 0.0319 - val_acc: 0.9482

**Plot for accuracy and loss on both training and validation**

In [ ]:
# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
#epochs = 5
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Checking model with drone captured images

In [ ]:
import os
#directory to store the images taken by drone
os.mkdir("testing_outputs")

In [ ]:
source = "/content/testing_outputs/"
from skimage.transform import resize
from skimage.transform import resize
classification=['broad','grass','soil','soya']
for filename in os.listdir(source):
  #changing path to current directory
  os.chdir(source)
  #extracting the path
  this_file = source+filename 
  #extracting the image name
  this_file = this_file[25:]
  #storing image to new_image
  new_image=plt.imread(this_file)
  print(this_file)
  
  #to resize the image to match model input size
  resized_image = resize(new_image, (150,150,3))
  #to predict the input image using our model
  predictions = model.predict(np.array( [resized_image] ))
  #list_index defines 0 for broadleaf, 1 for grass,2 for soil,3 for soybean
  list_index = [0,1,2,3]
  #storing predicted output by the model to x
  x = predictions
  # arranging the list in such a way that highest probability list index comes to first position of the list
  for i in range(4):
    for j in range(4):
      if x[0][list_index[i]] > x[0][list_index[j]]:
        temp = list_index[i]
        list_index[i] = list_index[j]
        list_index[j] = temp
  #Show the sorted labels in order from highest probability to lowest
  print(list_index,end='\t')
  print(classification[list_index[0]],end='\t')
  if(classification[list_index[0]]=='broad'):
    print('weed',end='\n')
    
    print('...........')

102.tif
[0, 1, 3, 2]	broad	weed
...........

1018.tif
[2, 1, 0, 3]	soil	non weed
...........

1032.tif
[3, 1, 0, 2]	soya	non weed
...........

101.tif
[3, 1, 0, 2]	soya	non weed
...........

106.tif
[0, 1, 3, 2]	broad	weed
...........

1.tif
[0, 1, 3, 2]	broad	weed
...........

3565.tif
[3, 1, 0, 2]	soya	non weed
...........

100.tif
[0, 1, 3, 2]	broad	weed
...........

4327.tif
[3, 1, 0, 2]	soya	non weed
...........

1025.tif
[0, 1, 3, 2]	broad	weed

Using "VGG16" model

In [ ]:
#importing dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow as tf
import os

In [ ]:
#building vgg16 model
model_vgg16 = tf.keras.applications.VGG16(include_top=False, input_shape=(150,150, 3))

Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
58892288/58889256 [==============================] - 1s 0us/step

In [ ]:
for layer in model_vgg16.layers:
    layer.trainable = False

In [ ]:
#activation function
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.3)
leaky_relu.__name__ = 'leaky_relu'
#flatten the layers
flat1 = tf.keras.layers.Flatten()(model_vgg16.layers[-1].output)
#256 neuron layers
class1 = tf.keras.layers.Dense(256, activation=leaky_relu)(flat1)
#using dropouts
drop1 = tf.keras.layers.Dropout(0.5)(class1)
class2 = tf.keras.layers.Dense(256, activation=leaky_relu)(drop1)
drop2 = tf.keras.layers.Dropout(0.5)(class2)
#using softmax function 
output = tf.keras.layers.Dense(4, activation='softmax')(drop2)

model_vgg16 = tf.keras.models.Model(inputs=model_vgg16.inputs, outputs=output)

In [ ]:
# model summary
model_vgg16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 37, 37, 256)       295168    
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 37, 37, 256)       590080    
_________________________________________________________________
block3_conv3 (Conv2D)        (None, 37, 37, 256)       590080    
_________________________________________________________________
block3_pool (MaxPooling2D)   (None, 18, 18, 256)       0         
_________________________________________________________________
block4_conv1 (Conv2D)        (None, 18, 18, 512)       1180160   
_________________________________________________________________
block4_conv2 (Conv2D)        (None, 18, 18, 512)       2359808   
_________________________________________________________________
block4_conv3 (Conv2D)        (None, 18, 18, 512)       2359808   
_________________________________________________________________
block4_pool (MaxPooling2D)   (None, 9, 9, 512)         0         
_________________________________________________________________
block5_conv1 (Conv2D)        (None, 9, 9, 512)         2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 9, 9, 512)         2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 9, 9, 512)         2359808   
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 4, 4, 512)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1028      
=================================================================
Total params: 16,878,916
Trainable params: 2,164,228
Non-trainable params: 14,714,688

In [ ]:
#compile the model
model_vgg16.compile(optimizer=keras.optimizers.Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train the model
history = model_vgg16.fit(train_gen, epochs=10,
                          validation_data=test_gen)

Epoch 1/10
192/192 [==============================] - 82s 427ms/step - loss: 0.3048 - accuracy: 0.9026 - val_loss: 0.0850 - val_accuracy: 0.9723
Epoch 2/10
192/192 [==============================] - 75s 393ms/step - loss: 0.1191 - accuracy: 0.9633 - val_loss: 0.0482 - val_accuracy: 0.9824
Epoch 3/10
192/192 [==============================] - 76s 394ms/step - loss: 0.1008 - accuracy: 0.9712 - val_loss: 0.0477 - val_accuracy: 0.9850
Epoch 4/10
192/192 [==============================] - 76s 395ms/step - loss: 0.0798 - accuracy: 0.9769 - val_loss: 0.0640 - val_accuracy: 0.9827
Epoch 5/10
192/192 [==============================] - 76s 395ms/step - loss: 0.0782 - accuracy: 0.9775 - val_loss: 0.1019 - val_accuracy: 0.9694
Epoch 6/10
192/192 [==============================] - 76s 394ms/step - loss: 0.0548 - accuracy: 0.9845 - val_loss: 0.0467 - val_accuracy: 0.9857
Epoch 7/10
192/192 [==============================] - 76s 395ms/step - loss: 0.0416 - accuracy: 0.9879 - val_loss: 0.0751 - val_accuracy: 0.9808
Epoch 8/10
192/192 [==============================] - 76s 396ms/step - loss: 0.0375 - accuracy: 0.9883 - val_loss: 0.0676 - val_accuracy: 0.9827
Epoch 9/10
192/192 [==============================] - 76s 397ms/step - loss: 0.0666 - accuracy: 0.9830 - val_loss: 0.0675 - val_accuracy: 0.9824
Epoch 10/10
192/192 [==============================] - 75s 393ms/step - loss: 0.0473 - accuracy: 0.9861 - val_loss: 0.0469 - val_accuracy: 0.9863

In [ ]:

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

Plot for accuracy and loss on both training and validation

In [ ]:
#plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Checking model with drone captured images

In [ ]:
import os
#directory to store the images taken by drone
os.mkdir("testing_outputs")

In [ ]:
source = "/content/testing_outputs/"
from skimage.transform import resize
from skimage.transform import resize
classification=['broad','grass','soil','soya']
for filename in os.listdir(source):
  #changing path to current directory
  os.chdir(source)
  #extracting the path
  this_file = source+filename 
  #extracting the image name
  this_file = this_file[25:]
  #storing image to new_image
  new_image=plt.imread(this_file)
  print(this_file)
  
  #to resize the image to match model input size
  resized_image = resize(new_image, (150,150,3))
  #to predict the input image using our model
  predictions = model_vgg16.predict(np.array( [resized_image] ))
  #list_index defines 0 for broadleaf, 1 for grass,2 for soil,3 for soybean
  list_index = [0,1,2,3]
  #storing predicted output by the model to x
  x = predictions
  # arranging the list in such a way that highest probability list index comes to first position of the list
  for i in range(4):
    for j in range(4):
      if x[0][list_index[i]] > x[0][list_index[j]]:
        temp = list_index[i]
        list_index[i] = list_index[j]
        list_index[j] = temp
  #Show the sorted labels in order from highest probability to lowest
  print(list_index,end='\t')
  print(classification[list_index[0]],end='\t')
  if(classification[list_index[0]]=='broad'):
    print('weed',end='\n')
    
    print('...................')
    

  else:
    print("non weed",end='\n')
    print('....................')  
    

102.tif
[0, 3, 1, 2]	broad	weed


1018.tif
[2, 0, 3, 1]	soil	non weed


1032.tif
[3, 0, 1, 2]	soya	non weed


101.tif
[3, 0, 2, 1]	soya	non weed


106.tif
[0, 3, 1, 2]	broad	weed


1.tif
[0, 3, 1, 2]	broad	weed


3565.tif
[3, 0, 1, 2]	soya	non weed


100.tif
[0, 3, 1, 2]	broad	weed


4327.tif
[3, 1, 0, 2]	soya	non weed


1025.tif
[0, 3, 1, 2]	broad	weed